# Interaction Tracking Analysis

Code to analyze the output from `interactionTracker.js`: process it to determine the likely interactions and reduce the amount of data, then produce some statistics.

In [155]:
import os
import glob

def get_newest_interaction_file(directory, startsWith = "interactions_"):
    # Get list of all files in the directory
    files = glob.glob(os.path.join(directory, '*'))
    files = [file.replace("\\", "/") for file in files]
    
    # Check if the directory is empty
    if not files:
        return None

    # Filter for interaction data.
    if startsWith:
        files = [file for file in files if file.split("/")[-1].startswith(startsWith)]
    
    # Get the newest file based on modification time
    newest_file = max(files, key=os.path.getmtime)
    return newest_file

In [156]:
import json

fileDir = 'C:/Users/rafb/AppData/Roaming/Code/User/workspaceStorage/b03f3379a1447d3620f3d4a069f1d983/grandFileNavigator.grandfilenavigator'
filePath = get_newest_interaction_file(fileDir)

interactionData = []
with open(filePath) as interactionDataFile:
    for interaction in interactionDataFile.readlines():
        interactionData.append(json.loads(interaction))

print(interactionData)

[{'timeStamp': 1743260361714, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': '', 'sourceRange': '', 'targetRange': '295-330'}, {'timeStamp': 1743260361717, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '295-330', 'targetRange': '296-330'}, {'timeStamp': 1743260362256, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '296-330', 'targetRange': '299-333'}, {'timeStamp': 1743260362264, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '299-333', 'targetRange': '303-333'}, {'timeStamp': 1743260362282, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '303-333', 'targetRange': '305-333'}, {'timeStamp': 1743260362313, 'interactio

In [157]:
def remove_erroneous(interactionData):
    fixedInteractionData = []

    for interaction in interactionData:
        if interaction["interactionType"] == "ChangeFile":
            if not interaction["targetFilePath"] or interaction["sourceFilePath"] == interaction["targetFilePath"]:
                continue
        fixedInteractionData.append(interaction)
    
    return fixedInteractionData


def remove_duplicates(interactionData):
    cleanedInteractionData = []

    i = 0
    while i < len(interactionData):
        if i == len(interactionData) - 1 or interactionData[i] != interactionData[i+1]:
            cleanedInteractionData.append(interactionData[i])
        i += 1
    
    return cleanedInteractionData

In [ ]:
import copy

# Merge scrolls together.
# A scroll is all changes of the visible ranges with less than 250 between them (i.e., covers short interruptions).


def detect_scrolling(interactionData):
    # max time in ms between ChangeVisibleRanges entries to still be considered part of one scroll
    maxTimeBetweenChanges = 250
    scrollingInteractionData = []

    changeRangesInteraction = None
    for interaction in interactionData:
        if interaction["interactionType"] != "ChangeVisibleRanges":
            if changeRangesInteraction is not None:
                scrollingInteractionData.append(changeRangesInteraction)
                changeRangesInteraction = None
            scrollingInteractionData.append(interaction)
            continue
        
        if changeRangesInteraction is None:
            changeRangesInteraction = copy.deepcopy(interaction)
            continue

        isScrollInteraction = changeRangesInteraction["interactionType"] == "Scroll"
        lastInteractionEndTime = changeRangesInteraction["endTime"] if isScrollInteraction else changeRangesInteraction["timeStamp"]
        if interaction["timeStamp"] - lastInteractionEndTime > maxTimeBetweenChanges:
            scrollingInteractionData.append(changeRangesInteraction)
            changeRangesInteraction = None
        else:
            if changeRangesInteraction["interactionType"] == "ChangeVisibleRanges": # wasn't treated as scroll yet
                changeRangesInteraction["interactionType"] = "Scroll"
                changeRangesInteraction["startTime"] = changeRangesInteraction["timeStamp"]
            changeRangesInteraction["endTime"] = interaction["timeStamp"]
            changeRangesInteraction["targetRange"] = interaction["targetRange"]
    
    if changeRangesInteraction is not None:
        scrollingInteractionData.append(changeRangesInteraction)

    return scrollingInteractionData

In [159]:
def combine_edits(interactionData):
    # max time in ms between ChangeVisibleRanges entries to still be considered part of one edit session
    maxTimeBetweenChanges = 2000
    editingInteractionData = []

    editInteraction = None
    for interaction in interactionData:
        if interaction["interactionType"] != "EditFile":
            if editInteraction is not None:
                editingInteractionData.append(editInteraction)
                editInteraction = None
            editingInteractionData.append(interaction)
            continue
        
        if editInteraction is None:
            editInteraction = copy.deepcopy(interaction)
            continue

        isEditingSession = editInteraction["interactionType"] == "EditingSession"
        lastInteractionEndTime = editInteraction["endTime"] if isEditingSession else editInteraction["timeStamp"]
        if interaction["timeStamp"] - lastInteractionEndTime > maxTimeBetweenChanges:
            editingInteractionData.append(editInteraction)
            editInteraction = None
        else:
            if editInteraction["interactionType"] == "EditFile": # wasn't treated as session yet
                editInteraction["interactionType"] = "EditingSession"
                editInteraction["startTime"] = editInteraction["timeStamp"]
            editInteraction["endTime"] = interaction["timeStamp"]

    if editInteraction is not None:
        editingInteractionData.append(editInteraction)

    return editingInteractionData

In [160]:
def process_jumps(interactionData):
    navigationInteractions = ["ClickJumpButton", "ClickStatusBar", "NavigationJump"]
    jumpInteractionData = []

    i = 0
    while i < len(interactionData):
        if interactionData[i]["interactionType"] in navigationInteractions:
            triggeredByUiInteraction = interactionData[i]["interactionType"] != "NavigationJump"
            jumpEntryIndex = i+2 if triggeredByUiInteraction else i+1

            combinedEntry = interactionData[jumpEntryIndex]
            combinedEntry["interactionType"] = "NavigationJump"
            combinedEntry["backwards"] = interactionData[i]["backwards"]

            if interactionData[i]["interactionType"] == "ClickJumpButton":
                combinedEntry["origin"] = "SidebarButton"
            elif interactionData[i]["interactionType"] == "ClickStatusBar":
                combinedEntry["origin"] = "StatusBar"
            else:
                combinedEntry["origin"] = "KeyboardShortcut"

            jumpInteractionData.append(combinedEntry)
            i += 3 if triggeredByUiInteraction else 2
        else:
            jumpInteractionData.append(interactionData[i])
            i += 1
        
    return jumpInteractionData

In [163]:
for interaction in interactionData:
    print(interaction)

refinedInteractionData = detect_scrolling(interactionData)
refinedInteractionData = combine_edits(refinedInteractionData)
refinedInteractionData = remove_erroneous(refinedInteractionData)
refinedInteractionData = remove_duplicates(refinedInteractionData)
refinedInteractionData = process_jumps(refinedInteractionData)

{'timeStamp': 1743260361714, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': '', 'sourceRange': '', 'targetRange': '295-330'}
{'timeStamp': 1743260361717, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '295-330', 'targetRange': '296-330'}
{'timeStamp': 1743260362256, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '296-330', 'targetRange': '299-333'}
{'timeStamp': 1743260362264, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '299-333', 'targetRange': '303-333'}
{'timeStamp': 1743260362282, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '303-333', 'targetRange': '305-333'}
{'timeStamp': 1743260362313, 'interactionType'

In [164]:
for interaction in refinedInteractionData:
    print(interaction)

{'timeStamp': 1743260361714, 'interactionType': 'Scroll', 'sourceFilePath': '', 'sourceRange': '', 'targetRange': '296-330', 'startTime': 1743260361714, 'endTime': 1743260361717}
{'timeStamp': 1743260362264, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '299-333', 'targetRange': '333-333', 'startTime': 1743260362264, 'endTime': 1743260362677}
{'timeStamp': 1743260363526, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '324-333', 'targetRange': '239-273', 'startTime': 1743260363526, 'endTime': 1743260365528}
{'timeStamp': 1743260379096, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\agent.py', 'sourceRange': '235-269', 'targetRange': '148-182', 'startTime': 1743260379096, 'endTime': 1743260380977}
{'timeStamp': 1743260382380, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users